In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest


In [3]:
data = {
    'Product Name': ['ROUTER 1', 'TRANSCIEVER', 'SWITCH 1', 'ACCESS POINT 1', 'ACCESS POINT 2',
                     'SWITCH 2', 'SWITCH 3', 'POWER SUPPLY 1', 'SWITCH 4', 'SWITCH 5',
                     'SWITCH 6', 'ACCESS POINT 3', 'SUPERVISOR ENGINE', 'SWITCH 7',
                     'WIRELESS CONTROLLER', 'SWITCH 8', 'SWITCH 9', 'ACCESS POINT 4',
                     'SWITCH 10', 'POWER SUPPLY 2'],
    'Demand_20Q4': [906, 87173, 3651, 0, 0, 13432, 28585, 115709, 0, 4571, 0, 65048,
                    0, 6073, 1208, 4, 6187, 0, 1227, 28279],
    'Demand_21Q1': [941, 102849, 10409, 0, 0, 12463, 19686, 98337, 270, 4100, 0, 67810,
                    108, 5459, 963, -4, 5301, 0, 24186, 22704],
    'Demand_21Q2': [663, 126375, 10575, 9576, 0, 13452, 44724, 147681, 850, 4830, 0, 90928,
                    1972, 6736, 1698, 36, 7172, 0, 7680, 24098],
    'Demand_21Q3': [769, 128629, 17934, 46029, 0, 14581, 37781, 185327, 2717, 5472, 0, 148547,
                    2427, 12213, 1833, 305, 7582, 0, 16772, 31461],
    'Demand_21Q4': [3565, 179125, 15104, 109452, 26769, 15831, 31905, 194170, 3833, 7391, 222,
                    228945, 2841, 13883, 2423, 1093, 10964, 0, 0, 47009],
    'Demand_22Q1': [1020, 138750, 15525, 54955, 33670, 13864, 37888, 220386, 4279, 7894, 1549,
                    204184, 2096, 27045, 2602, 2360, 11501, 0, 12186, 35145],
    'Demand_22Q2': [1670, 208760, 6938, 60478, 74853, 15012, 32689, 204604, 6089, 7101, 4619,
                    245453, 2096, 15162, 2341, 4317, 14980, 21667, 7680, 45978],
    'Demand_22Q3': [38, 116126, 9595, 62708, 80944, 11349, 30442, 194538, 8063, 6943, 4764,
                    245803, 2096, 13855, 1830, 4343, 9158, 30498, 16772, 34518],
    'Demand_22Q4': [2141, 150803, 9231, 60478, 0, 10422, 27028, 185160, 2535, 6126, 5015,
                    136283, 2096, 12351, 1567, 4094, 11601, 40505, 0, 40605],
    'Demand_23Q1': [1716, 82163, 12000, 62708, 76192, 8876, 21246, 147284, 6700, 6000, 5887,
                    76020, 3157, 8933, 1210, 2756, 3788, 36932, 13992, 26925],
    'Demand_23Q2': [1716, 82408, 11820, 62708, 113704, 9812, 21149, 136997, 5878, 6632, 11525,
                    51586, 2910, 5909, 1659, 3126, 5338, 58652, 16324, 23635],
    'Demand_23Q3': [1948, 67132, 11184, 62708, 93200, 8194, 10584, 100623, 8605, 3499, 7207,
                    53431, 2867, 6643, 1311, 2808, 5595, 41942, 11746, 17799],
    'Demand_Planners_Forecast_23Q4': [2141, 85000, 12000, 62771, 76192, 8499, 13500, 119759, 6700,
                                      6000, 5877, 52305, 3157, 6208, 1438, 3700, 5700, 36932, 13992, 26500],
    'Marketing_Teams_Forecast_23Q4': [2865, 141285, 11820, 79113, 113704, 9812, 27066, 142486, 5878,
                                       6632, 11525, 104907, 2910, 14552, 1689, 3672, 5164, 58652, 16324, 27742],
    'Stat_ML_Teams_Forecast_23Q4': [1948, 97995, 11184, 65310, 93200, 8194, 16960, 116208, 8605,
                                     3499, 7207, 63623, 2867, 8260, 1539, 3506, 5942, 41942, 11746, 24978]
}
df = pd.DataFrame(data)

In [4]:
df['Growth_Q4_Q1'] = (df['Demand_21Q1'] - df['Demand_20Q4']) / df['Demand_20Q4']
df['Growth_Q1_Q2'] = (df['Demand_21Q2'] - df['Demand_21Q1']) / df['Demand_21Q1']
df['Growth_Q2_Q3'] = (df['Demand_21Q3'] - df['Demand_21Q2']) / df['Demand_21Q2']
df['Growth_Q3_Q4'] = (df['Demand_21Q4'] - df['Demand_21Q3']) / df['Demand_21Q3']

df['Moving_Avg_4Q'] = df[['Demand_21Q1', 'Demand_21Q2', 'Demand_21Q3', 'Demand_21Q4']].mean(axis=1)
df['Moving_Avg_8Q'] = df[['Demand_20Q4', 'Demand_21Q1', 'Demand_21Q2', 'Demand_21Q3',
                         'Demand_21Q4', 'Demand_22Q1', 'Demand_22Q2', 'Demand_22Q3']].mean(axis=1)



In [5]:
# Features and Target
X = df[['Demand_20Q4', 'Demand_21Q1', 'Demand_21Q2', 'Demand_21Q3', 'Demand_21Q4', 
        'Demand_22Q1', 'Demand_22Q2', 'Demand_22Q3', 'Demand_22Q4', 'Demand_23Q1', 
        'Demand_23Q2', 'Demand_23Q3']]
y = df['Demand_Planners_Forecast_23Q4']


# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


In [6]:
iso_forest = IsolationForest(contamination=0.1)  # Assuming 10% of data are outliers
outlier_labels = iso_forest.fit_predict(X_normalized)

In [7]:
X_cleaned = X_normalized[outlier_labels == 1]
y_cleaned = y[outlier_labels == 1]

In [8]:
linear_model = LinearRegression()
lasso_model = Lasso(alpha=0.1) 
ridge_model = Ridge(alpha=0.1)  


linear_model.fit(X_cleaned, y_cleaned)
lasso_model.fit(X_cleaned, y_cleaned)
ridge_model.fit(X_cleaned, y_cleaned)

C:\Users\ayv44\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.616e+06, tolerance: 1.198e+06
  model = cd_fast.enet_coordinate_descent(


Ridge(alpha=0.1)

In [9]:
df['Forecasted_23Q4_Linear'] = linear_model.predict(X_normalized)
df['Forecasted_23Q4_Lasso'] = lasso_model.predict(X_normalized)
df['Forecasted_23Q4_Ridge'] = ridge_model.predict(X_normalized)

# Calculate R-squared and RMSE for each model
r_squared_linear = r2_score(y, df['Forecasted_23Q4_Linear'])
rmse_linear = sqrt(mean_squared_error(y, df['Forecasted_23Q4_Linear']))

r_squared_lasso = r2_score(y, df['Forecasted_23Q4_Lasso'])
rmse_lasso = sqrt(mean_squared_error(y, df['Forecasted_23Q4_Lasso']))

r_squared_ridge = r2_score(y, df['Forecasted_23Q4_Ridge'])
rmse_ridge = sqrt(mean_squared_error(y, df['Forecasted_23Q4_Ridge']))

print("Linear Regression Model Results:")
print("R-squared:", r_squared_linear)
print("RMSE:", rmse_linear)
print("\nLasso Regression Model Results:")
print("R-squared:", r_squared_lasso)
print("RMSE:", rmse_lasso)

print("\nRidge Regression Model Results:")
print("R-squared:", r_squared_ridge)
print("RMSE:", rmse_ridge)

Linear Regression Model Results:
R-squared: 0.9727485749588805
RMSE: 5461.401315915585

Lasso Regression Model Results:
R-squared: 0.9657552605841172
RMSE: 6122.1845184145095

Ridge Regression Model Results:
R-squared: 0.9475282994690593
RMSE: 7578.3045012414195


In [10]:
print(df[['Product Name', 'Forecasted_23Q4_Linear']])

           Product Name  Forecasted_23Q4_Linear
0              ROUTER 1             2313.091573
1           TRANSCIEVER            85032.340287
2              SWITCH 1            11177.267699
3        ACCESS POINT 1            62873.213944
4        ACCESS POINT 2            76106.516672
5              SWITCH 2             9028.078810
6              SWITCH 3            13590.946687
7        POWER SUPPLY 1           123371.849154
8              SWITCH 4             7035.410241
9              SWITCH 5             5278.395010
10             SWITCH 6             6710.684916
11       ACCESS POINT 3            76392.430960
12    SUPERVISOR ENGINE             3281.600424
13             SWITCH 7             6543.830170
14  WIRELESS CONTROLLER             1656.363294
15             SWITCH 8             3203.012969
16             SWITCH 9             5031.758468
17       ACCESS POINT 4            36919.762235
18            SWITCH 10            14194.259247
19       POWER SUPPLY 2            26330

In [11]:
print(df[['Product Name', 'Forecasted_23Q4_Lasso']])

           Product Name  Forecasted_23Q4_Lasso
0              ROUTER 1            2423.378664
1           TRANSCIEVER           85023.021893
2              SWITCH 1           10938.722002
3        ACCESS POINT 1           62928.039915
4        ACCESS POINT 2           76094.869703
5              SWITCH 2            9179.482397
6              SWITCH 3           13637.121344
7        POWER SUPPLY 1          120441.623373
8              SWITCH 4            6861.584409
9              SWITCH 5            5446.264868
10             SWITCH 6            6637.083214
11       ACCESS POINT 3           79611.137490
12    SUPERVISOR ENGINE            3090.400858
13             SWITCH 7            6574.391200
14  WIRELESS CONTROLLER            1676.333580
15             SWITCH 8            3189.060345
16             SWITCH 9            5185.926190
17       ACCESS POINT 4           36952.671680
18            SWITCH 10           14278.017081
19       POWER SUPPLY 2           26190.630659


In [12]:
print(df[['Product Name', 'Forecasted_23Q4_Ridge']])

           Product Name  Forecasted_23Q4_Ridge
0              ROUTER 1            2175.514639
1           TRANSCIEVER           85099.257245
2              SWITCH 1           11152.910233
3        ACCESS POINT 1           62527.748827
4        ACCESS POINT 2           76058.682464
5              SWITCH 2            9106.972926
6              SWITCH 3           14725.789184
7        POWER SUPPLY 1          122108.148935
8              SWITCH 4            6658.992482
9              SWITCH 5            5288.501434
10             SWITCH 6            6748.206581
11       ACCESS POINT 3           85947.195809
12    SUPERVISOR ENGINE            2826.673259
13             SWITCH 7            7935.059043
14  WIRELESS CONTROLLER            1680.941984
15             SWITCH 8            3108.143438
16             SWITCH 9            5966.636829
17       ACCESS POINT 4           37103.750744
18            SWITCH 10           13522.359891
19       POWER SUPPLY 2           24620.858797
